In [ ]:
%load_ext autoreload
%autoreload 2
from matplotlib import pyplot as plt
import text_extract as text
import json
import os
import re

In [ ]:
JSON_PATH = "./settings.json"

with open(JSON_PATH) as f:
    settings = json.load(f)

file_settings = settings["file"]
file_names = text.data.get_file_names(file_settings)

image_array = []
h_crop_array = []
v_crop_array = []
h_max_size = 0
v_max_size = 0

for name in file_names:
    path = file_settings["path"]
    gray_img = text.data.read_image(path, name)

    rotate_img = text.extract.set_textbox_horizontally(gray_img, settings)
    textbox_img = text.extract.extract_textbox(rotate_img, settings)
    enhanced_img = text.extract.enhance_textbox(textbox_img, settings)

    char_crop_settings = settings["cropPoints"]
    [h_crop_points, h_max] = text.data.get_char_info(enhanced_img, char_crop_settings, mode="h")
    [v_crop_points, v_max] = text.data.get_char_info(enhanced_img, char_crop_settings, mode="v")

    image_array.append(enhanced_img)

    h_crop_array.append(h_crop_points)
    v_crop_array.append(v_crop_points)

    h_max_size = h_max if h_max > h_max_size else h_max_size
    v_max_size = v_max if v_max > v_max_size else v_max_size

# plt.imshow(image_array[0][:200, :200])
# plt.show()

In [ ]:
# create empty data
# max size (20, 11)

text_len = (len(v_crop_array[0]) - 1) * (len(h_crop_array[0]) - 1)
char_array = np.zeros((text_len, v_max_size, h_max_size), dtype=image_array[0].dtype)

char_idx = 0
for v_idx in range(len(v_crop_array[0]) - 1):
    for h_idx in range(len(h_crop_array[0]) - 1):
        empty_img = np.zeros((v_max_size, h_max_size), dtype=image_array[0].dtype)
        v_start = v_crop_array[0][v_idx]
        h_start = h_crop_array[0][h_idx]
        v_next = v_crop_array[0][v_idx + 1]
        h_next = h_crop_array[0][h_idx + 1]
        char_img = image_array[0][v_start:v_next, h_start:h_next]

        empty_img[:char_img.shape[0], :char_img.shape[1]] = char_img
        char_array[char_idx, :, :] = empty_img
        char_idx += 1

text_array = make_random_text(1, text_len)

# for i in range(20):
#     plt.figure(figsize=(0.5, 0.5))
#     plt.imshow(char_array[i, :, :])
#     plt.show()

In [ ]:
import tensorflow as tf
from tensorflow import keras

X_train_full = char_array
y_train_full = text_array

X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[v_max_size, h_max_size]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])

# model.summary()

model.compile(loss=keras.losses.sparse_categorical_crossentropy, 
    optimizer=keras.optimizers.SGD(),
    metrics=[keras.metrics.sparse_categorical_accuracy]
    )

history = model.fit(X_train, y_train, epochs=30, 
                    validation_data=(X_valid, y_valid))


In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
# 헷갈릴 글자 대상들
# 0 Γ
# 8 Δ
# 5 Σ
# $ -> § or 직접 폰트 수정
#  (공백) Ч
# ` §

# 절대로 헷갈리지 않을 만한 문자들
# 그리스어
# Γ Δ Ξ Σ